In [110]:
import matplotlib
import pandas as pd
import requests
import io
import numpy as np
import matplotlib.pyplot as plt
import math
import datetime
import re
from datetime import datetime


%matplotlib inline
    


%matplotlib inline
url1 = "https://raw.githubusercontent.com/docju/datasciencecapstone/main/results.csv"
download1 = requests.get(url1).content

df = pd.read_csv(io.StringIO(download1.decode('ISO-8859-1')))
df['date']=pd.to_datetime(relevant_data['DateTime'])
df['outcome']=df['FTHG']-df['FTAG']
all_results=df[['Season','date','HomeTeam','AwayTeam','FTHG','FTAG','outcome']][~df1['Season'].isin(['2020-21','2021-22'])]

In [147]:

covid_season=all_results[all_results['Season']=='2019-20']
covid_season_pre_break=covid_season[covid_season['date']<=datetime(2020, 3, 20)]
covid_season_post_break=covid_season[covid_season['date']>datetime(2020, 3, 20)]

print(covid_season_post_break)

        Season                date          HomeTeam          AwayTeam  FTHG  \
10332  2019-20 2020-06-17 18:00:00       Aston Villa  Sheffield United     0   
10333  2019-20 2020-06-17 20:15:00          Man City           Arsenal     3   
10334  2019-20 2020-06-19 18:00:00           Norwich       Southampton     0   
10335  2019-20 2020-06-19 20:15:00         Tottenham        Man United     1   
10336  2019-20 2020-06-20 12:30:00           Watford         Leicester     1   
10337  2019-20 2020-06-20 15:00:00          Brighton           Arsenal     2   
10338  2019-20 2020-06-20 17:30:00          West Ham            Wolves     0   
10339  2019-20 2020-06-20 19:45:00       Bournemouth    Crystal Palace     0   
10340  2019-20 2020-06-21 14:00:00         Newcastle  Sheffield United     3   
10341  2019-20 2020-06-21 16:15:00       Aston Villa           Chelsea     1   
10342  2019-20 2020-06-21 19:00:00           Everton         Liverpool     0   
10343  2019-20 2020-06-22 20:00:00      

In [185]:
df2=covid_season_pre_break.copy()
output = np.asmatrix(df2.groupby(['HomeTeam', 'AwayTeam'])['outcome'].max().unstack())

df3=covid_season_post_break.copy()
output_post = np.asmatrix(df3.groupby(['HomeTeam', 'AwayTeam'])['outcome'].max().unstack())



FileNotFoundError: [Errno 2] No such file or directory: 'output.p'

In [172]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=500):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0]
    n_movies = ratings_mat.shape[1]
    num_ratings = np.count_nonzero(~np.isnan(ratings_mat)) #-n_users
    
    # initialize the user and movie matrices with random values
    home_mat = np.random.rand(n_users, latent_features)
    away_mat = np.random.rand(latent_features, n_movies)
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # header for running results
    print("Optimizaiton Statistics")
    print("Iterations | Mean Squared Error ")
    #print(home_mat)
    #print(away_mat)
    # for each iteration
    for iteration in range(iters):

        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):
 
                
                # if the rating exists
                if pd.isna(ratings_mat[i,j]) is False:
                    
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    diff = ratings_mat[i, j] - np.dot(home_mat[i, :], away_mat[:, j])
                    
                    # Keep track of the sum of squared errors for the matrix
                    sse_accum += diff**2
                    
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        home_mat[i, k] += learning_rate * (2*diff*away_mat[k, j])
                        away_mat[k, j] += learning_rate * (2*diff*home_mat[i, k])

        # print results for iteration
        if iteration in {0,1,iters-1}:
            print("%d \t\t %f" % (iteration+1, sse_accum / num_ratings))
        
    return home_mat, away_mat 

In [180]:
home_mat,away_mat=FunkSVD(output,latent_features=10,learning_rate=0.005, iters=500)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 6.504899
2 		 4.163364
500 		 0.004662


In [187]:
print(np.dot(home_mat, away_mat)-output_post)


[[             nan              nan              nan              nan
               nan              nan              nan              nan
   -2.02395742e+00  -2.15967040e+00              nan              nan
               nan  -1.66509454e+00              nan              nan
               nan  -2.28035438e+00              nan              nan]
 [ -3.22473242e+00              nan              nan              nan
               nan   7.29416133e-01  -1.59648762e+00              nan
               nan              nan              nan   2.22242431e+00
               nan              nan  -6.25244831e-01              nan
               nan              nan              nan   3.43193038e-01]
 [             nan              nan              nan              nan
               nan              nan   7.89430244e+00              nan
   -1.30497962e+01              nan              nan              nan
    1.29201326e+01              nan              nan  -3.47757969e+00
    3.41390950e+00